In [1]:
%%capture
!python3 -m pip install pandas numpy matplotlib tensorflow tqdm bs4

In [2]:
import requests
import urllib3
from urllib.parse import urlparse, urljoin
import socket

from bs4 import BeautifulSoup
import chardet
import re
import json

import math
import statistics

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras import Sequential
import tensorflow.keras.layers
import tensorflow.keras.losses

from sklearn.model_selection import train_test_split

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

2023-03-25 18:59:37.772267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-25 18:59:37.772293: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import warnings
warnings.filterwarnings(action="ignore", module="urllib3")

In [4]:
raw_data = pd.read_csv("./raw_data.csv")
raw_data

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [5]:
raw_data["attempted_download"] = False

In [6]:
# def generate_alternative_urls(data_in):
#     original_url = data_in
#     url_with_scheme = "http://"+original_url if original_url[0:4] != "http" else original_url
#     parsed_url = urlparse(url_with_scheme)
#     scheme = parsed_url.scheme
#     host = parsed_url.netloc
#     path = parsed_url.path
#     query = parsed_url.query

#     a = np.empty((8, 8))
#     a.fill(np.nan)
#     alternative_urls = pd.DataFrame(a, columns=["url", "type", "content", "http", "path", "query"])
#     print(alternative_urls)
    
#     # Encoding: [ (scheme == "http"), (host != ""), (path != ""), (query != "") ]
    
#     alternative_urls[0,""] = "http://"+host
#     if(path != ""):
#         alternative_urls[0, 1, 1, 0] = "http://"+host+path
#     if(query != ""):
#         alternative_urls[0, 1, 0, 1] = "http://"+host+"?"+query
#     if(path != "" and query != ""):
#         alternative_urls[0, 1, 1, 1] = "http://"+host+path+"?"+query
        
#     alternative_urls[1, 1, 0, 0] = "https://"+host
#     if(path != ""):
#         alternative_urls[1, 1, 1, 0] = "https://"+host+path
#     if(query != ""):
#         alternative_urls[1, 1, 0, 1] = "https://"+host+"?"+query
#     if(path != "" and query != ""):
#         alternative_urls[1, 1, 1, 1] = "https://"+host+path+"?"+query
        
#     print(alternative_urls)
#     return alternative_urls

# #generate_alternative_urls(raw_data.loc[0, "url"])
# #raw_data.loc[:20, "url"].apply(generate_alternative_urls)
# generate_alternative_urls("http://test1.com/test2?test3")

In [7]:
MAX_QUERIES = 20
TIMEOUT = (5, 5)

In [8]:
def get_words(row_index):
    global raw_data
    content = raw_data.loc[row_index, "content"]
    words = re.compile('[\W_]+').sub(' word_', BeautifulSoup(content, 'html.parser').get_text()).split(" ")
    
    new_columns = [ word for word in set(words) if word not in raw_data.columns.to_list() and word != "word_" ]
    df = pd.DataFrame(np.zeros((raw_data.shape[0], len(new_columns))), columns=new_columns)
    raw_data = pd.concat((raw_data, df), axis=1)
                      
    for word in words:
        if(word != "word_"):
            raw_data.loc[row_index, word] = raw_data.loc[row_index, word] + 1

In [9]:
def get_location(ip_addr=None, hostname=None):
    try: 
        if(ip_addr is None and hostname is not None):
            ip_addr = socket.gethostbyname(hostname)
        if(not ip_addr in location_database):
            location_database[ip_addr] = requests.get("https://ipinfo.io/"+ip_addr+"/json").json()
    except socket.error:
        return {}
    return location_database[ip_addr]

In [10]:
def get_aux_data(row_index):
    global raw_data
    content = raw_data.loc[row_index, "content"]
    original_url = raw_data.loc[row_index, "url"]
    
    aux = []
    
    soup = BeautifulSoup(content, 'html.parser')
    for a in soup.find_all('a'):
        href = a.get('href')
        if(href is not None):
            url = get_absolute_url_from_relative(original_url, href)
            if(url is not None):
                hostname = urlparse(url).netloc
                json = get_location(hostname=hostname)
                
                try:
                    columns = [
                        "hostname_"+hostname,
                        "ip_addr_"+json["ip"],
                        "loc_"+json["loc"],
                        "city_"+json["city"],
                        "region_"+json["region"],
                        "country_"+json["country"],
                        "org_"+json["org"]
                    ]
                except KeyError:
                    print("Problem with JSON: ", json)
                    columns = []
                
                columns = [ re.compile('[\W_]+').sub('_', column) for column in columns ]
                
                aux = aux + columns
                
#                 new_columns = [ column for column in set(columns) if column not in raw_data.columns.to_list() ]
#                 df = pd.DataFrame(np.zeros((raw_data.shape[0], len(new_columns))), columns=new_columns)
#                 raw_data = pd.concat((raw_data, df), axis=1)
                
#                 for column in columns:
#                     raw_data.loc[row_index, column] = raw_data.loc[row_index, column] + 1

    raw_data.loc[row_index, "aux"] = " ".join(aux)

In [11]:
def get_absolute_url_from_relative(original_url, new_url):
    if(new_url.startswith("javascript")):
        return None
    if(new_url.startswith("http")):
        return new_url
    url_with_scheme = "http://"+original_url if not original_url.startswith("http") else original_url
    parsed_url = urlparse(url_with_scheme)
    url_scheme = parsed_url.scheme
    url_host = parsed_url.netloc
    
    return urljoin(url_scheme+"://"+url_host, new_url)

In [12]:
# def download_aggregate_location_data(row):
#     original_url = row["url"]
#     content = row["content"]
#     location = tuple(row[["city", "region", "country"]])
    
#     if(content == np.nan):
#         return [0, None, None, None, 0]
#     try:
#         soup = BeautifulSoup(content, 'html.parser')
#     except:
#         return [0, None, None, None, 0]
    
#     location_data = []
#     for a in soup.find_all('a'):
#         href = a.get('href')
#         if(href is not None):
#             url = get_absolute_url_from_relative(original_url, href)
#             if(url is not None):
#                 host = urlparse(url).netloc
#                 l = get_location_data(host)
#                 if(l != (None, None, None)):
#                     location_data.append(l)
             
    
#     location_data_without_original_location = [ l for l in location_data if l != location ]
#     if(len(location_data_without_original_location) > 0):
        
#         mode_location_data = statistics.mode(location_data_without_original_location)
        
#         mode_city = mode_location_data[0]
#         mode_region = mode_location_data[1]
#         mode_country = mode_location_data[2]
        
#         num_unique_locations = len(set(location_data_without_original_location))
    
#         return [len(location_data), mode_city, mode_region, mode_country, num_unique_locations]
    
#     else:
#         return [len(location_data), None, None, None, 0]

In [13]:
def download_data(num_queries=MAX_QUERIES, timeout=TIMEOUT):
    global raw_data
    start = 0
    if(len(raw_data[raw_data["attempted_download"] == True].index) != 0):
        start = raw_data[raw_data["attempted_download"] == True].index[-1]+1
    end = start + num_queries
    end = end if len(raw_data["url"]) > end else len(raw_data["url"])
    
    print("Downloading %d more rows ([%d:%d])" % (num_queries, start, end))
    for row_index in tqdm(range(start, end)):
        raw_data.loc[row_index, "attempted_download"] = True

        original_url = raw_data.loc[row_index, "url"]
        url_with_scheme = "http://"+original_url if not original_url.startswith("http") else original_url
        parsed_url = urlparse(url_with_scheme)
        hostname = parsed_url.netloc
        try:
            raw_data.loc[row_index, "hostname"] = hostname
            
            ip_addr = socket.gethostbyname(hostname)
            raw_data.loc[row_index, "ip_addr"] = ip_addr
            
            r = requests.get(original_url, verify=False, timeout=timeout)

            raw_data.loc[row_index, "status_code"] = r.status_code
            encoding = chardet.detect(r.content)['encoding']
            if(encoding == None):
                continue
            try:
                content = r.content.decode(encoding)
            except:
                continue
            raw_data.loc[row_index, "content"] = content

            redirects = 0
            for r_history in r.history:
                if(r_history.status_code == 301):
                    redirects = redirects + 1
            raw_data.loc[row_index, "redirects"] = redirects
            
            json = get_location(ip_addr=ip_addr)
            try:
                raw_data.loc[row_index, "loc"] = json["loc"]
                raw_data.loc[row_index, "city"] = json["city"]
                raw_data.loc[row_index, "region"] = json["region"]
                raw_data.loc[row_index, "country"] = json["country"]
                raw_data.loc[row_index, "org"] = json["org"]
                
            except KeyError:
                print("Problem with JSON: ", json)
                continue
                
            get_aux_data(row_index)
            
            #get_words(row_index)
            raw_data.loc[row_index, "words"] = " ".join(re.compile('[\W_]+').sub(' ', BeautifulSoup(content, 'html.parser').get_text()).split(" "))
            
            
            
        except socket.error:
            pass

In [14]:
def start():
    global raw_data, location_database
    try:
        with open("./location_data.json", 'r') as file:
            location_database = json.load(file)
        file.close()
    except FileNotFoundError:
        location_database = {}

    try:
        raw_data = pd.read_csv("./data.csv", index_col=0)
        if(MAX_QUERIES > 0):
            download_data()
            #raw_data = raw_data.copy()
            raw_data.to_csv("./data.csv")
            with open("./location_data.json", 'w') as file:
                json.dump(location_database, file)
            file.close()

    except FileNotFoundError:
        download_data()
        #raw_data = raw_data.copy()
        raw_data.to_csv("./data.csv")
        with open("./location_data.json", 'w') as file:
            json.dump(location_database, file)
        file.close()

    data = raw_data.copy()
    data = data.dropna()
    data = data.drop("attempted_download", axis=1)
    data = data[data["status_code"] == 200]
    data = data.drop("status_code", axis=1)
    data = data.reset_index(drop=True)
    return data

In [15]:
data = start()
data

/tmp/ipykernel_91200/2637865377.py:1: DtypeWarning: Columns (4,5,7,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  data = start()


  0%|          | 0/20 [00:00<?, ?it/s]

,url,type,hostname,ip_addr,content,redirects,loc,city,region,country,org,words,aux
0,http://larcadelcarnevale.com/catalogo/palloncini,defacement,larcadelcarnevale.com,185.53.177.31,"<!DOCTYPE html>\n<html data-adblockkey=""MFwwDQ...",0.0,"48.1374,11.5755",Munich,Bavaria,DE,AS19905 Neustar Security Services,larcadelcarnevale com Buy this domain larcade...,hostname_secure_voodoo_com ip_addr_192_64_146_...
1,https://docs.google.com/spreadsheet/viewform?f...,phishing,docs.google.com,172.253.115.102,"<!DOCTYPE html><html lang=""en"" class=""HB1eCd-U...",2.0,"38.8951,-77.0364",Washington,"Washington, D.C.",US,AS15169 Google LLC,Sorteo Notebook Diciembre 2012JavaScript isn t...,hostname_accounts_google_com ip_addr_142_251_1...
2,http://interpark.com/displaycorner/FreeMarket....,benign,interpark.com,211.233.74.23,"\r\n\r\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTM...",0.0,"37.5660,126.9784",Seoul,Seoul,KR,AS3786 LG DACOM Corporation,Shopper s Heaven 인터파크 홈 리빙 세탁 청소용품 리빙 최신 등록순 ...,hostname_interpark_com ip_addr_211_233_74_23 l...
3,http://www.pn-wuppertal.de/links/2-linkseite/5...,defacement,www.pn-wuppertal.de,217.160.0.93,"\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",1.0,"49.0094,8.4044",Karlsruhe,Baden-Wurttemberg,DE,AS8560 IONOS SE,Links KontaktAnfahrtDatenschutzImpressum Home...,hostname_www_pn_wuppertal_de ip_addr_217_160_0...
4,http://www.parafiapiaski.pl/index.php?option=c...,defacement,www.parafiapiaski.pl,185.253.212.22,<!doctype html>\n<html>\n\n<head>\n\n<meta htt...,0.0,"52.2298,21.0118",Warsaw,Mazovia,PL,AS48707 AS48707 OPS PL sp. z o.o.,AfterMarket pl domena parafiapiaski pl Domena...,hostname_www_aftermarket_pl ip_addr_185_253_21...
5,http://fanpage.gr/family/gineka/%ce%b1%cf%85%c...,benign,fanpage.gr,104.26.8.146,"<!DOCTYPE html>\n<html lang=""el"">\n<head>\n<me...",3.0,"37.7621,-122.3971",San Francisco,California,US,"AS13335 Cloudflare, Inc.",Αυτοί είναι οι 13 ωραιότεροι Έλληνες Αντέχεις...,hostname_www_fanpage_gr ip_addr_104_26_8_146 l...
6,http://worldoftanks.ru/ru/content/guide/paymen...,benign,worldoftanks.ru,92.223.22.65,"<!DOCTYPE html>\n<html lang=""ru"">\n <head>\...",1.0,"49.5806,6.1118",Luxembourg,Luxembourg,LU,AS199524 G-Core Labs S.A.,Перенос аккаунта Wargaming ВЫБОР РЕГИОНА ДЛЯ ...,hostname_adn_wargaming_net ip_addr_92_223_23_2...
7,http://9779.info/%E5%84%BF%E7%AB%A5%E7%AB%8B%E...,malware,9779.info,160.153.63.37,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",0.0,"33.4484,-112.0740",Phoenix,Arizona,US,"AS398101 GoDaddy.com, LLC",儿童立体纸贴画图片展示 贴画网 树叶粘贴画 豆子粘贴画 剪贴画 拼贴画 布贴画 儿童贴画 ...,hostname_www_9779_info ip_addr_160_153_63_37 l...
8,http://correios.com.br/para-sua-empresa/comuni...,benign,correios.com.br,201.48.198.80,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",0.0,"-15.8333,-47.9000",Brasília,Federal District,BR,AS16735 ALGAR TELECOM S/A,Correios Navegação Toggle navigation Página I...,hostname_www_correios_com_br ip_addr_201_48_19...
9,http://chelurao.ru/index.php?option=com_conten...,defacement,chelurao.ru,91.226.31.82,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 S...",0.0,"52.3740,4.8897",Amsterdam,North Holland,NL,AS49981 WorldStream B.V.,Chelurao ru Chelurao ru Сайт в процессе разра...,hostname_i7_ru ip_addr_91_226_31_57 loc_52_374...


In [16]:
#AUTOTUNE = tf.data.AUTOTUNE

#train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [17]:
feature_names = ["words", "aux", "city", "region", "country"]
X = [data[feature_name] for feature_name in feature_names]

sl = tf.keras.layers.StringLookup(output_mode='one_hot')
sl.adapt(data["type"])

y = sl(list(data["type"]))

X_train, X_test, y_train, y_test = train_test_split(data[feature_names], y, test_size=0.33)

X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.33)

2023-03-25 18:59:53.130264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-25 18:59:53.130315: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-25 18:59:53.130342: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benjamin-XPS-13-9370): /proc/driver/nvidia/version does not exist
2023-03-25 18:59:53.130592: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 18:59:53.184863: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of th

In [19]:
max_features = 10000
sequence_length = 250

def custom_standardization(input_data):
    return tf.strings.lower(input_data)

def get_vectorize_layer(column):
    vectorize_layer = tf.keras.layers.TextVectorization(
        standardize=custom_standardization,
        max_tokens=max_features,
        output_mode='int',
        output_sequence_length=sequence_length
    )
    vectorize_layer.adapt(data[column])
    return vectorize_layer

def get_string_lookup(column):
    lookup = tf.keras.layers.StringLookup(
        output_mode='one_hot',
        max_tokens=sequence_length,
        pad_to_max_tokens=True,
    )
    lookup.adapt(data[column])
    return lookup

#inputs = {}
#for feature_name in feature_names:
#    inputs[feature_name] = tf.keras.Input(shape=(1,), dtype=tf.string, name=feature_name)

inputs = tf.keras.Input(shape=(len(feature_names),), dtype=tf.string)
outputs = []
for i in range(len(feature_names)):
    feature_name = feature_names[i]
    flatten = tf.keras.layers.Flatten()
    embedding_layer = tf.keras.layers.Embedding(max_features, 5)
    if(feature_name == "words" or feature_name == "aux"):
        layer = get_vectorize_layer(feature_name)
    else:
        layer = get_string_lookup(feature_name)
    print(inputs[:, i])
    outputs.append(embedding_layer(layer(flatten(inputs[:,i]))))
    
    
#inputs = tf.concat(list(inputs.values()), axis=-1)
outputs = tf.concat(outputs, axis=-1)
print(outputs.shape[1:])

sequential_model = tf.keras.Sequential([
    #tf.keras.Input(shape=outputs.shape[1:]),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.GlobalAveragePooling1D(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])


model = tf.keras.Model(inputs=inputs, outputs=sequential_model(outputs) )
model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])

model.summary()
sequential_model.summary()


model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=25, batch_size=1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None), name='tf.__operators__.getitem_4/strided_slice:0', description="created by layer 'tf.__operators__.getitem_4'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None), name='tf.__operators__.getitem_6/strided_slice:0', description="created by layer 'tf.__operators__.getitem_6'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None), name='tf.__operators__.getitem_8/strided_slice:0', description="created by layer 'tf.__operators__.getitem_8'")
(250, 25)
Model: "model"
__________________________________

In [20]:
def to_type(one_hot_encoding):
    return sl.get_vocabulary()[np.argmax(tf.nn.softmax(one_hot_encoding))]

predictions = [ to_type(model.predict(X_test)[i]) for i in range(len(X_test)) ]
actuals = [ to_type(y_test[i]) for i in range(len(y_test)) ]

pd.DataFrame(np.array([actuals, predictions]).T, columns=["actual", "predicted"])

,actual,predicted
0,malware,malware
1,defacement,defacement
2,defacement,defacement
3,defacement,phishing
4,malware,malware
5,malware,malware
6,defacement,defacement
7,defacement,defacement
8,benign,phishing
9,benign,defacement


## TODO
1. Run this on the entire raw_data set
2. Do one-hot-encoding for the city, region, country, and type (probably using tensorflow/keras/sklearn)
3. Do text encoding on url and content (probably using tensorflow/keras/sklearn)
4. Split training and testing (using tensorflow/keras/sklearn)
5. Implement Z-Fold cross-validation (probably using tensorflow/keras/sklearn)
6. Train CNN using tensorflow/keras/sklearn
7. Optinally, fine tune hyperparameters (e.g. epochs, activation function, layers, etc.)
8. Graph results